In [77]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d
import pandas as pd
import os


In [3]:
file_path = 'datasets/json/box_plant1/box_plant1_frame300_400.json'
# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Now 'data' is a dictionary
num_key = len(data.keys())
if num_key > 0:
    print(f"Data loaded, number of frames = {num_key}")

Data loaded, number of frames = 100


In [4]:
frame = 350
points = np.array(data[str(frame)])

In [42]:
# Convert numpy array to open3d point cloud
def quickVisualize(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])
    return

In [81]:
def loadData(path):
    # List all files in the specified path, ignoring directories
    df = pd.read_csv(path)
    points_xyz = df.to_numpy()
    return points_xyz[:,8:11]

In [70]:
from preprocess import listFiles
### for handling entier point cloud
def shortPassFilter(points, threshold = 10):
    """ Filter points that are further than a distance away"""
    distance = (points[:,0]**2 + points[:,1]**2 + points[:,2]**2)**0.5
    mask = distance < threshold
    return points[mask]

# prepare path
directory = r'datasets/json/box_plant1/'
files = listFiles(directory)
files.sort()

# prepare frame 0 as reference
path = directory + files[0]
with open(path, 'r') as file:
    data = json.load(file)
pts_frame_0 = np.array(data[str(0)])
pts_frame_1 = np.array(data[str(20)])
pts_frame_0 = shortPassFilter(pts_frame_0, 7)
pts_frame_1 = shortPassFilter(pts_frame_1, 7)


In [90]:
### For handling manual point clouds
path1 = r'datasets/box_plant1_manual_frame0.csv'
path2 = r'datasets/box_plant1_manual_frame60.csv'
pts_frame_0 = loadData(path1)
pts_frame_60 = loadData(path2)

In [91]:
quickVisualize(pts_frame_0)

In [93]:
# getTransformations()


# Create Open3D PointCloud objects
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pts_frame_0)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pts_frame_60)

# Preprocess point clouds: Downsample for faster ICP
voxel_size = 0.05
pcd1_down = pcd1.voxel_down_sample(voxel_size)
pcd2_down = pcd2.voxel_down_sample(voxel_size)

# Estimate normals for ICP
radius_normal = voxel_size * 2
pcd1_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
pcd2_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))

# Apply ICP registration
threshold = 0.1  # Maximum correspondence points-pair distance
trans_init = np.identity(4)  # Initial transformation

print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(pcd1_down, pcd2_down, threshold, trans_init)
print(evaluation)

# Perform point-to-plane ICP
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    pcd1_down, pcd2_down, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

print("Transformation is:")
print(reg_p2l.transformation)

# Apply the transformation to the original point cloud
pcd1.transform(reg_p2l.transformation)

# Visualize the aligned point clouds
o3d.visualization.draw_geometries([pcd1, pcd2])
# o3d.visualization.draw_geometries([pcd1])

Initial alignment
RegistrationResult with fitness=1.369327e-01, inlier_rmse=6.199265e-02, and correspondence_set size of 175
Access transformation to get result.
Apply point-to-plane ICP
Transformation is:
[[ 0.74325888 -0.64520892  0.17683803  0.81929044]
 [ 0.66841548  0.70510816 -0.23673451 -3.51065646]
 [ 0.02805328  0.2941563   0.95534553  0.1334603 ]
 [ 0.          0.          0.          1.        ]]
